# Homework week4

In [ ]:
!pip freeze | grep scikit-learn

In [9]:
import pickle
import pandas as pd
import numpy as np

## Q1. Notebook

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
year = 2022
month = 2
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
np.std(y_pred)

5.28140357655334

## Q2. Preparing the output

In [14]:
!mkdir output

In [11]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [15]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['y_pred'] = y_pred

df_result.to_parquet(
    'output/result.pkl',
    engine='pyarrow',
    compression=None,
    index=False
)

In [18]:
! ls -lah output/result.pkl

-rw-rw-r-- 1 hirobo hirobo 58M Jun 17 23:48 output/result.pkl
